In [1]:
import numpy as np

# LiteBIRD's IMO
import json

# opening IMO schema.json file and interpreting it as a dictionary
data = json.load(open('/afs/mpa/temp/monelli/litebird/litebird_imo-master/IMO/schema.json',))  

In the next cell I build up a list of dictionaries. Each element in the list is a dictionary that contains all the relevant information associated to one of the detectors of the M1-140 channel.

In [2]:
# looking into the IMO, data['data_files'] is where the relevant info is stored 
data_files = data['data_files']

# counting how many objects are in data_files
nkey=0
for key in data_files:
    nkey = nkey+1

# looking for the detectors belonging to the M1-140 channel
for i in range(nkey):
    test = data_files[i]
    if(test['name'] == 'channel_info'):
        metadata = test['metadata']
        if(metadata['channel'] == 'M1-140'):
            detector_names = metadata['detector_names']
            break

ndet = len(detector_names)
det_indices = range(ndet)
print('the M1-140 channel has '+str(ndet)+' detectors')

list_of_dictionaries = []       

# looking for the metadata of the detectors in detector_names
for d in detector_names:
    for j in range(nkey):
        test = data_files[j]
        if(test['name'] == 'detector_info' and test['metadata']['name'] == d):
            list_of_dictionaries.append(test['metadata']) # this list contains all the info about the M1-140 detectors
            break

the M1-140 channel has 366 detectors


Some of the keys have the same value for all the dictionaries (i.e., for all the detectors in the channel). I built up an array for each key that is not "constant", in order to better work with them.

In [3]:
# the following quantities are actually identical for each detector
# (I checked that by means of the all_equals function defined below)
fwhm = list_of_dictionaries[0]['fwhm_arcmin']
ellipticity = list_of_dictionaries[0]['ellipticity'] # it's zero, IS THIS REALISTIC?
bandcenter = list_of_dictionaries[0]['bandcenter_ghz']
bandwidth = list_of_dictionaries[0]['bandwidth_ghz']
sampling_freq = list_of_dictionaries[0]['sampling_rate_hz']
net = list_of_dictionaries[0]['net_ukrts']
pol_sensitivity = list_of_dictionaries[0]['pol_sensitivity_ukarcmin']
fknee = list_of_dictionaries[0]['fknee_mhz']
fmin = list_of_dictionaries[0]['fmin_hz']
alpha = list_of_dictionaries[0]['alpha']

def all_equals(string):
    test = np.empty(ndet,dtype=object)
    for i in det_indices:
        test[i] = list_of_dictionaries[i][string]
    return all(x==test[0] for x in test)
# example of usage: print(all_equals('alpha')

# instead, the following change detector by detector
pol_array = np.empty(ndet,dtype=object)
orient_array = np.empty(ndet,dtype=object)
quat_array = np.empty((ndet,4))

for i in det_indices:
    pol_array[i] = list_of_dictionaries[i]['pol']
    orient_array[i] = list_of_dictionaries[i]['orient']
    quat_array[i] = np.array(list_of_dictionaries[i]['quat'])

Here I'm printing the offset quaternions associated to a pair of T/B detectors. Shouldn't they be identical?

In [4]:
print('detector '+str(list_of_dictionaries[0]['name'])+': '+str(quat_array[0]))
print('detector '+str(list_of_dictionaries[1]['name'])+': '+str(quat_array[1]))

detector M02_030_QA_140T: [-0.08355164 -0.14339497  0.          0.98613235]
detector M02_030_QA_140B: [-0.16047549 -0.04231562  0.69730087  0.69730087]


In [7]:
print(orient_array)

['Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'U' 'U' 'Q' 'Q' 'U' 'U' 'Q' 'Q'
 'Q' 'Q' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Q' 'Q'
 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q'
 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q'
 'U' 'U' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q'
 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q'
 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q'
 'Q' 'Q' 'U' 'U' 'U' 'U' 'U' 'U' 'Q' 'Q' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U'
 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q'
 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'Q' 'Q'
 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q'
 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q' 'U' 'U'
 'U' 'U' 'Q' 'Q' 'Q' 'Q' 'U' 'U' 'U' 'U' 'Q' 'Q' 'Q